In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, Dataset
from matplotlib import pyplot as plt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
cifar_train = torchvision.datasets.CIFAR10('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                             ]))

cifar_test = torchvision.datasets.CIFAR10('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                             ]))

### Generating artificially partial labelled dataset

In [ ]:
def generate_partial_labelled(dataset, r, p) :

    # List to store the inputs in numpy array format
    x = []

    # List to store the outputs in numpy array format
    y = []

    for i in range(len(dataset)) :

        original_label = dataset[i][1]

        labels = []
        labels.append(original_label)

        if np.random.rand() <= p :
            for j in range(r) :
                # Find a label other than the original one and append it
                new_label = np.random.randint(0, 10)
                while new_label in labels :
                    new_label = np.random.randint(0, 10)
                
                labels.append(new_label)
        
        x.append(dataset[i][0].detach().numpy().reshape(3, 32, 32))

        # One hot encoding the labels before appending to the y list
        one_hot = np.zeros(10, dtype='float32')

        for label in labels :
            one_hot[label] = 1

        # Regularization
        one_hot = np.sum(one_hot)

        y.append(one_hot)

    return np.array(x), np.array(y)

In [ ]:
# Creating custom dataset and dataloader
class cifar10_partial_dataset(Dataset) :
    def __init__(self, dataset, r, p) :
        self.x, self.y = generate_partial_labelled(dataset, r, p)
        self.length = len(self.x)


    def __getitem__(self, index) :
        return self.x[index], self.y[index]

    def __len__(self) :
        return self.length

In [ ]:
r = 1
p = 0.1

In [ ]:
train_dataset = cifar10_partial_dataset(cifar_train, r, p)
train_dataloader = DataLoader(train_dataset, 32, shuffle=True)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.main = nn.Sequential(
            nn.ConvTranspose2d( 110, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d( 64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d( 64 * 2, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64 * 4, 11, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
D = Discriminator().to(device)
G = Generator().to(device)

D.apply(weights_init)
G.apply(weights_init)

criterion = nn.MSELoss()

# optimizer
D_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
G_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
D_optimizer = torch.optim.Adam(D.parameters(), lr = 0.0002)
G_optimizer = torch.optim.Adam(G.parameters(), lr = 0.0002)

In [ ]:
def train_D(x, y) :
    D.zero_grad()

    batch_size = len(x)

    # Training on real data
    x_real, y_real = x.view(-1, 3, 32, 32), torch.concat((y, torch.zeros(batch_size, 1)), 1)
    x_real, y_real = x_real.to(device), y_real.to(device)

    D_output = D(x_real).view(-1, 11)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # Training of fake data
    z = torch.concat((torch.randn(batch_size, 100), y), 1).view(-1, 110, 1, 1).to(device)

    x_fake = G(z)
    y_fake = torch.concat((torch.zeros(batch_size, 10), torch.ones(batch_size, 1)), 1)
    y_fake = y_fake.to(device)

    D_output = D(x_fake).view(-1, 11)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()


def train_G(x, y) :
    G.zero_grad()

    batch_size = len(x)

    z = torch.concat((torch.randn(batch_size, 100), y), 1).view(-1, 110, 1, 1).to(device)
    y_real = torch.concat((y, torch.zeros(batch_size, 1)), 1).to(device)

    G_output = G(z)

    D_output = D(G_output).view(-1, 11)
    G_loss = criterion(D_output, y_real)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

In [ ]:
n_epoch = 30
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, y) in enumerate(train_dataloader):
        G_losses.append(train_G(x, y))
        D_losses.append(train_D(x, y))

    print(f'{epoch}/{n_epoch}: loss_d: {torch.mean(torch.FloatTensor(D_losses))}, loss_g: {torch.mean(torch.FloatTensor(G_losses))}')

In [ ]:
with torch.no_grad() :
    y = torch.zeros(1,10)
    y[0][7] = 1
    z = torch.concat((torch.randn(1, 100), y), 1).view(-1, 110, 1, 1).to(device)

    output = G(z)
    expected = torch.zeros(1, 11)
    expected[0][2] = 1

    # print(D(output))
    # print(torch.sum((expected-D(output).cpu().detach())**2))

    output = output.cpu().detach().numpy()
    output = output.reshape(3, 32, 32)

    reshapedImg = np.zeros((32, 32, 3), dtype='float32')

    reshapedImg[:, :, 0] = output[0, :, :]
    reshapedImg[:, :, 1] = output[1, :, :]
    reshapedImg[:, :, 2] = output[2, :, :]

    plt.imshow(reshapedImg)
    plt.show()

In [ ]:
torch.save(D.state_dict(), f'./models/cifar10_disc_r_value_1_p_value_{p}.pt')
torch.save(G.state_dict(), f'./models/cifar10_gen_r_value_1_p_value_{p}.pt')